In [1]:
import sys, os, re, json, time

import pandas as pd
import pickle
import h5py

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import plotting
from PIL import Image
from tqdm import tqdm
from utils import imread, img_data_2_mini_batch, imgs2batch

from sklearn import metrics
from sklearn.metrics import accuracy_score

# from naive import EncDec
from attention import EncDec as FuseAttEncDec
# from rnn_att import EncDec
from data_loader import VQADataSet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
N = 5000
dataset_filename = "./data/data_{}.pkl".format(N)
dataset = None
print(dataset_filename)
if (os.path.exists(dataset_filename)):
    with open(dataset_filename, 'rb') as handle:
        print("reading from " + dataset_filename)
        dataset = pickle.load(handle)
else:
    dataset = VQADataSet(Q=N)
    with open(dataset_filename, 'wb') as handle:
        print("writing to " + dataset_filename)
        pickle.dump(dataset, handle)

assert(dataset is not None)
def debug(v,q,a):
    print('\nV: {}\nQ: {}\nA: {}'.format(v.shape, q.shape, a.shape))


./data/data_10000.pkl


100%|██████████| 1658/1658 [00:00<00:00, 49976.69it/s]


VQADataSet init time: 37.085965156555176
writing to ./data/data_10000.pkl


In [3]:
embed_size        = 256
hidden_size       = 256
batch_size        = 50
ques_vocab_size   = len(dataset.vocab['question'])
ans_vocab_size    = len(dataset.vocab['answer'])
num_layers        = 1
n_epochs          = 30
learning_rate     = 0.001
momentum          = 0.98
attention_size    = 26
debug             = False

print(ques_vocab_size, ans_vocab_size)

3356 1658


In [4]:
def eval_model(data_loader, model, criterion, optimizer, batch_size, training=False,
              epoch = 0, total_loss_over_epochs=[], scores_over_epochs=[]):
    running_loss = 0.
    final_labels, final_preds = [], []
    scores, losses = [], []
    if data_loader is None:
        return
    
    run_type = None
    if training:
        run_type = 'train'
        model.train()
    else:
        run_type = 'test'
        model.eval()
    
    for i, minibatch in enumerate(data_loader):
        # extract minibatch
        t0 = time.time()
        idxs, v, q, a, q_len = minibatch
        
        # convert torch's DataLoader output to proper format.
        # torch gives a List[Tensor_1, ... ] where tensor has been transposed. 
        # batchify transposes back.`
        v = v.to(device)
        q = VQADataSet.batchify_questions(q).to(device)
        a = a.to(device)

        logits = model(v, q, q_len)
        preds = torch.argmax(logits, dim=1)

#         loss = criterion(logits, a)
        loss = F.nll_loss(logits, a)
        running_loss += loss.item()
        
#         score = metrics.precision_recall_fscore_support(preds.tolist(),
#                                                         a.tolist(),
#                                                         average='weighted')
        score = metrics.accuracy_score(preds.tolist(),a.tolist())
    
        scores.append(score)
        losses.append(loss)
        
        loss_key = '{}_loss'.format(run_type)
        total_loss_over_epochs['{}_loss'.format(run_type)].append(loss)
        scores_over_epochs['{}_scores'.format(run_type)].append(score)
        
        if training and optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
   
        final_labels += a.tolist()
        final_preds  += preds.tolist()
        if i%10==0:
            score = np.mean(scores)
            print("Epoch {}: {} Loss: {} Score: {} t: {}".format(epoch, run_type,loss, score, time.time()-t0))
#             plotting.plot_score_over_n_epochs(scores_over_epochs, score_type='precision', fig_size=(7,3))
#             plotting.plot_loss_over_n_epochs(total_loss_over_epochs, hard_key=loss_key, fig_size=(7, 3))
            
    return running_loss, final_labels, final_preds

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = EncDec(embed_size, hidden_size, ques_vocab_size, ans_vocab_size, rnn_layers).to(device)
model = FuseAttEncDec(embed_size, hidden_size, attention_size, 
                      ques_vocab_size, ans_vocab_size, num_layers, debug).to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.get_parameters(), lr=learning_rate, momentum=momentum)
# optimizer = torch.optim.Adam(model.get_parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loader = dataset.build_data_loader(train=True, args={'batch_size': batch_size})
test_loader  = dataset.build_data_loader(test=True, args={'batch_size': batch_size})

best_score = 0

train_all_loss, train_all_labels, train_all_preds = [], [], []
print("model built, start training.")
total_loss_over_epochs, scores_over_epochs = plotting.get_empty_stat_over_n_epoch_dictionaries()
total_loss_over_epochs2, scores_over_epochs2 = plotting.get_empty_stat_over_n_epoch_dictionaries()
for epoch in tqdm(range(n_epochs)):
    t0= time.time()
    tr_loss, tr_labels, tr_preds = eval_model(data_loader = train_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = optimizer,
                                     batch_size  = batch_size,
                                     training    = True,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs,
                                     scores_over_epochs     = scores_over_epochs)
    
    tr_loss, ts_labels, ts_preds = eval_model(data_loader = test_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = None,
                                     batch_size  = batch_size,
                                     training    = False,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs2,
                                     scores_over_epochs     = scores_over_epochs2)
    
    
    score = metrics.accuracy_score(ts_preds,ts_labels)
#     total_loss_over_epochs['train_loss'].append(tr_loss)
#     scores_over_epochs['train_scores'].append(train_scores)
    
#     if True:# or epoch%1 == 0:
    print("\n"+"#==#"*7 + "epoch: {}".format(epoch) + "#==#"*7)
    print('TEST ACC: {}'.format(score))
    print("#==#"*7 + "time: {}".format(time.time()-t0) + "#==#"*7 + "\n")
#         print(train_scores)
#     plotting.plot_score_over_n_epochs(scores_over_epochs, score_type='precision', fig_size=(8,5))
#     plotting.plot_loss_over_n_epochs(total_loss_over_epochs, fig_size=(8, 5), title="Loss")
    
    
    
    

  0%|          | 0/30 [00:00<?, ?it/s]

batch_size: 50 shuffle: True
batch_size: 50 shuffle: False
model built, start training.
Epoch 0: train Loss: 7.39285945892334 Score: 0.0 t: 0.3493924140930176
Epoch 0: train Loss: 5.687780380249023 Score: 0.12000000000000002 t: 0.31960606575012207
Epoch 0: train Loss: 5.581486701965332 Score: 0.14190476190476192 t: 0.32895612716674805
Epoch 0: train Loss: 5.432222843170166 Score: 0.15548387096774194 t: 0.3270988464355469
Epoch 0: train Loss: 4.512545108795166 Score: 0.16439024390243903 t: 0.32564568519592285
Epoch 0: train Loss: 5.237973690032959 Score: 0.16627450980392156 t: 0.34931063652038574
Epoch 0: train Loss: 4.351251125335693 Score: 0.16721311475409834 t: 0.32316040992736816
Epoch 0: train Loss: 5.537405490875244 Score: 0.1712676056338028 t: 0.32485294342041016
Epoch 0: train Loss: 4.251253128051758 Score: 0.17135802469135802 t: 0.32932615280151367
Epoch 0: train Loss: 4.707014560699463 Score: 0.17076923076923078 t: 0.32668113708496094
Epoch 0: train Loss: 5.310774326324463 Sco

  3%|▎         | 1/30 [06:16<3:01:51, 376.26s/it]


#==##==##==##==##==##==##==#epoch: 0#==##==##==##==##==##==##==#
TEST ACC: 0.2677006068779501
#==##==##==##==##==##==##==#time: 376.2606565952301#==##==##==##==##==##==##==#

Epoch 1: train Loss: 4.891851425170898 Score: 0.14 t: 0.32692670822143555
Epoch 1: train Loss: 4.433326244354248 Score: 0.18727272727272723 t: 0.3278353214263916
Epoch 1: train Loss: 4.934946060180664 Score: 0.18666666666666668 t: 0.33093976974487305
Epoch 1: train Loss: 4.339747428894043 Score: 0.2 t: 0.32828855514526367
Epoch 1: train Loss: 3.859689950942993 Score: 0.20146341463414633 t: 0.3282618522644043
Epoch 1: train Loss: 4.790663719177246 Score: 0.19999999999999998 t: 0.3303954601287842
Epoch 1: train Loss: 4.880555629730225 Score: 0.20360655737704922 t: 0.32822322845458984
Epoch 1: train Loss: 4.5336012840271 Score: 0.20760563380281688 t: 0.32785463333129883
Epoch 1: train Loss: 3.7525463104248047 Score: 0.20814814814814814 t: 0.33019089698791504
Epoch 1: train Loss: 4.6499505043029785 Score: 0.209010989

  7%|▋         | 2/30 [11:55<2:50:24, 365.18s/it]

...

 37%|███▋      | 11/30 [50:39<1:21:39, 257.85s/it]


#==##==##==##==##==##==##==#epoch: 10#==##==##==##==##==##==##==#
TEST ACC: 0.3256911665542819
#==##==##==##==##==##==##==#time: 284.0714313983917#==##==##==##==##==##==##==#

Epoch 11: train Loss: 4.89210319519043 Score: 0.1 t: 0.32849860191345215
Epoch 11: train Loss: 4.061110973358154 Score: 0.22363636363636366 t: 0.32921838760375977
Epoch 11: train Loss: 4.151802062988281 Score: 0.23809523809523808 t: 0.32776355743408203
Epoch 11: train Loss: 3.8922600746154785 Score: 0.2387096774193548 t: 0.3278687000274658
Epoch 11: train Loss: 3.6495134830474854 Score: 0.23463414634146343 t: 0.331920862197876
Epoch 11: train Loss: 3.2331125736236572 Score: 0.23764705882352943 t: 0.3323657512664795
Epoch 11: train Loss: 4.094050407409668 Score: 0.23508196721311475 t: 0.32981276512145996
Epoch 11: train Loss: 4.37645149230957 Score: 0.23746478873239435 t: 0.3314225673675537
Epoch 11: train Loss: 4.183826923370361 Score: 0.23703703703703707 t: 0.32846546173095703
Epoch 11: train Loss: 3.6482915878

 40%|████      | 12/30 [54:50<1:16:44, 255.79s/it]


#==##==##==##==##==##==##==#epoch: 11#==##==##==##==##==##==##==#
TEST ACC: 0.32231962238705325
#==##==##==##==##==##==##==#time: 250.98848271369934#==##==##==##==##==##==##==#

Epoch 12: train Loss: 3.958484172821045 Score: 0.2 t: 0.3277127742767334
Epoch 12: train Loss: 3.5834085941314697 Score: 0.23454545454545456 t: 0.3294503688812256
Epoch 12: train Loss: 3.7957804203033447 Score: 0.25142857142857145 t: 0.3303647041320801
Epoch 12: train Loss: 4.3805766105651855 Score: 0.2387096774193548 t: 0.3283576965332031
Epoch 12: train Loss: 3.849226474761963 Score: 0.24439024390243902 t: 0.3286471366882324
Epoch 12: train Loss: 3.8507158756256104 Score: 0.2447058823529412 t: 0.33100223541259766
Epoch 12: train Loss: 3.7199268341064453 Score: 0.23901639344262293 t: 0.33179140090942383
Epoch 12: train Loss: 3.752009630203247 Score: 0.23746478873239435 t: 0.33176112174987793
Epoch 12: train Loss: 4.437532424926758 Score: 0.23185185185185186 t: 0.33189916610717773
Epoch 12: train Loss: 4.48808

KeyboardInterrupt: 

In [ ]:
for epoch in range(1):
    ts_loss, ts_labels, ts_preds = eval_model(data_loader = test_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = None,
                                     batch_size  = batch_size,
                                     training    = False,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs2,
                                     scores_over_epochs     = scores_over_epochs2)
    score = metrics.accuracy_score(ts_preds,ts_labels)
    print("ACC: " + str(score))

In [ ]:
print(tr_labels[0])
print(tr_preds[0])